# Fine-tuning LLMs

Originally based on https://github.com/ShawhinT/YouTube-Blog/tree/main/LLMs/fine-tuning
But could not fork since it is just one notebook in a large collection of project in single git.

The following is a short tutorial on fine tuning an LLM using mainly Huggin Face (HF) helper functions and PyTorch

The **dataset** import is from HF [https://pypi.org/project/datasets/]  
This gives access to HF public datasets and your own uploaded datasets.  
Create and load datasets: [https://huggingface.co/docs/datasets/upload_dataset]  
Find more datasets: [https://huggingface.co/datasets]  

The **transformers** import is also from HF [https://pypi.org/project/transformers/]  
Similar to the datasets class this gives you access to HF hosted models and just like with datasets you can upload and host your own ones.  
Upload and share models: [https://huggingface.co/docs/hub/en/models-uploading]
Find more models: [https://huggingface.co/models]


In [6]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

### dataset

In [2]:
# # how dataset was generated

# # load imdb data
# imdb_dataset = load_dataset("imdb")

# # define subsample size
# N = 1000 
# # generate indexes for random subsample
# rand_idx = np.random.randint(24999, size=N) 

# # extract train and test data
# x_train = imdb_dataset['train'][rand_idx]['text']
# y_train = imdb_dataset['train'][rand_idx]['label']

# x_test = imdb_dataset['test'][rand_idx]['text']
# y_test = imdb_dataset['test'][rand_idx]['label']

# # create new dataset
# dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),
#                              'validation':Dataset.from_dict({'label':y_test,'text':x_test})})

In [8]:
# load dataset
dataset = load_dataset('shawhin/imdb-truncated')
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

### model

**distilbert-base-uncased**  
66M parameters  
"This model is a distilled version of the BERT base model"  
"The model was trained on 8 16 GB V100 for 90 hours"  
https://huggingface.co/distilbert/distilbert-base-uncased  

**AutoModelForSequenceClassification.from_pretrained()**  
Will in this case download and cache the model from HF model repo

If you want to clone models and run from local instead:  
go to: https://huggingface.co/bert-base-uncased?clone=true  (For this example)  
Clone the model, then do similar but with local path:  

_model_checkpoint = "/path/to/your/local/model"  
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint ..._  



In [9]:
model_checkpoint = 'distilbert-base-uncased'

# define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 483/483 [00:00<00:00, 1.24MB/s]
model.safetensors: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 268M/268M [00:22<00:00, 11.7MB/s]
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
# display architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### preprocess data

In [10]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28.0/28.0 [00:00<00:00, 84.0kB/s]
vocab.txt: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 232k/232k [00:00<00:00, 3.72MB/s]
tokenizer.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 466k/466k [00:00<00:00, 1.51MB/s]


In [11]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [12]:
# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 7797.57 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [13]:
# create data collator
# this will dynamically pad examples in each batch to be equal length
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### evaluation

**evaluate**: is another package from HF  
https://pypi.org/project/evaluate/  


In [14]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [16]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

### Apply untrained model to text

In [18]:
# define list of examples
text_list = [
    "It was good.", 
    "Not a fan, don't recommed.", 
    "Better than the first one.", 
    "This is not worth watching even once.", 
    "This one is a pass."
]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")  # pt = pytorch https://huggingface.co/docs/transformers/en/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__.return_tensors
    
    # compute logits
    logits = model(inputs).logits
    
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - Positive
Not a fan, don't recommed. - Positive
Better than the first one. - Positive
This is not worth watching even once. - Positive
This one is a pass. - Positive


### Train model

#### PEFT - Parameter-Efficient Fine-Tuning
https://huggingface.co/docs/peft/en/index  
Another usefull HF package!

The package contains LoRA and similar algorithms used for finetuning models on simpler hardware.
https://huggingface.co/docs/peft/main/en/package_reference/lora  

Conceptual explanations for some of them can be found here:  
https://huggingface.co/docs/peft/en/conceptual_guides/adapter  

and all methods in the left side panel under **_API REFERENCES / ADAPTERS_**

See papers related to algos in PEFT:  
https://huggingface.co/collections/PEFT/peft-papers-6573a1a95da75f987fb873ad







In [20]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

**r** - **Dimensions in the LoRA trainble parameters, AxR * RxB = AxB**  
**lora_alpha** - The alpha parameter for Lora scaling. **When applying AxB to original W, its done with a scaling: W + (AxB * (lora_alpha/r))**  
**lora_dropout** - The dropout probability for Lora layers.  
**target_modules** - The names of the modules to apply Lora to.  

In [21]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [23]:
# Create a trainable model from froozen model + peft config
model = get_peft_model(model, peft_config)

model.print_trainable_parameters

# Print trainable layers
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name)

base_model.model.base_model.model.distilbert.transformer.layer.0.attention.q_lin.lora_A.default.weight
base_model.model.base_model.model.distilbert.transformer.layer.0.attention.q_lin.lora_B.default.weight
base_model.model.base_model.model.distilbert.transformer.layer.1.attention.q_lin.lora_A.default.weight
base_model.model.base_model.model.distilbert.transformer.layer.1.attention.q_lin.lora_B.default.weight
base_model.model.base_model.model.distilbert.transformer.layer.2.attention.q_lin.lora_A.default.weight
base_model.model.base_model.model.distilbert.transformer.layer.2.attention.q_lin.lora_B.default.weight
base_model.model.base_model.model.distilbert.transformer.layer.3.attention.q_lin.lora_A.default.weight
base_model.model.base_model.model.distilbert.transformer.layer.3.attention.q_lin.lora_B.default.weight
base_model.model.base_model.model.distilbert.transformer.layer.4.attention.q_lin.lora_A.default.weight
base_model.model.base_model.model.distilbert.transformer.layer.4.attentio

In [24]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [25]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [26]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# On my personal pc with 2x rtx 3090 this takes about TODO min

# On my eghed laptop with i7-1355U (10 cores) it said 1h30m 
# but after interupting at 52 batches, the printed prediction still gave 5/5 correct!

# so retraining 1% of the network, with 52 batches (2% of intended) still showed that it works!

# train model
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

### Generate prediction

In [32]:
print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt")

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
tensor([0])
It was good. - Negative
tensor([0])
Not a fan, don't recommed. - Negative
tensor([1])
Better than the first one. - Positive
tensor([0])
This is not worth watching even once. - Negative
tensor([1])
This one is a pass. - Positive


### Option 1: push model to hub

In [ ]:
# option 1: notebook login
from huggingface_hub import notebook_login
notebook_login() # ensure token gives write access

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

hf_name = 'TODO' # your hf username or org name
model_id = hf_name + "/" + model_checkpoint + "-lora-text-classification" # you can name the model whatever you want

model.push_to_hub(model_id) # save model
trainer.push_to_hub(model_id) # save trainer

### Option 2: save locally

In [ ]:
# Specify the local directory where you want to save the model
local_model_path = "/path/to/save/model"

# Save the model locally
model.save_pretrained(local_model_path)